# Data Preprocessing

Date Format Standardization Function

In [ ]:
def custom_date_parser(date_str):
    date_str = str(date_str).strip()  # Remove leading/trailing spaces
    try:
        # If format is d/m/Y
        if '/' in date_str:
            return pd.to_datetime(date_str + ' ' + '00:00:00')
        # If format is ISO 8601 with 'T' and 'Z'
        elif 'T' in date_str and 'Z' in date_str:
            return pd.to_datetime(date_str, format='%Y-%m-%dT%H:%MZ')
        # Try a general conversion
        else:
            return pd.to_datetime(date_str)
    except:
        return pd.NaT  # Return NaT if conversion fails

Data Preprocessing and Merging by Year (Flight and Weather)

In [ ]:
import pandas as pd
import glob

# Paths
# Upload all raw flight csv files of a specific year under a 'flight' folder
flight_path = 'flight/*.csv'
# Upload raw weather csv file of a specific year under a 'weather' folder
weather_path = 'weather/21_weather.csv'

# Load and sort flight CSV files
csv_files = sorted(glob.glob(flight_path))
dfs = []

# Process each flight CSV
for file in csv_files:
    df = pd.read_csv(file, skiprows=7)  # Skip first 7 rows
    df = df.iloc[:-1]  # Drop last row

    # Rename columns
    df.rename(columns={'Scheduled departure time': 'Scheduled Departure Time', 'Actual departure time': 'Actual Departure Time', 'Departure delay (Minutes)': 'Departure Delay (Minutes)'}, inplace=True)

    # Convert datetime for sorting and merging
    df['Scheduled Departure Datetime'] = pd.to_datetime(df['Date (MM/DD/YYYY)'] + ' ' + df['Scheduled Departure Time'])

    # Round to nearest previous hour for merging
    df['Rounded Datetime'] = df['Scheduled Departure Datetime'].dt.floor('h')

    # Sort by datetime
    df = df.sort_values(by='Scheduled Departure Datetime')

    dfs.append(df)

# Combine all flight data
flights = pd.concat(dfs, ignore_index=True)

# Load weather data
weather = pd.read_csv(weather_path)

# Convert weather date column to datetime
weather['Rounded Datetime'] = weather['date'].apply(custom_date_parser)

# Drop unnecessary columns (if any)
weather = weather.drop(columns=['date', 'station', 'precip_6hour'])  # Keep only relevant features

# Set negative precipitation values to 0
weather['precip_1hour'] = weather['precip_1hour'].apply(lambda x: max(x, 0))

# Fill missing numerical values using interpolation
interpolation_cols = ['air_temp', 'dew_temp', 'sea_level_pressure', 'wind_direction', 'wind_speed_rate', 'sky_condition', 'precip_1hour']
weather[interpolation_cols] = weather[interpolation_cols].interpolate(method='linear')  # Smooth filling
weather[interpolation_cols] = weather[interpolation_cols].fillna(method='bfill')  # Fill remaining with backward filling

# Normalize numerical columns using Min-Max scaling
weather[interpolation_cols] = (weather[interpolation_cols] - weather[interpolation_cols].min()) / (weather[interpolation_cols].max() - weather[interpolation_cols].min())

# Rename columns
weather.rename(columns={'air_temp': 'Air Temperature', 'dew_temp': 'Dew Temperature', 'sea_level_pressure': 'Sea Level Pressure', 'wind_direction': 'Wind Direction', 'wind_speed_rate': 'Wind Speed Rate', 'sky_condition': 'Sky Condition', 'precip_1hour': 'Precip 1 Hour'}, inplace=True)

# Merge flight data with weather data on the rounded datetime
merged_df = pd.merge(flights, weather, on='Rounded Datetime', how='left')

# Save the final merged data
merged_df.to_csv('2021.csv', index=False)

# Display first few rows
merged_df.head()

<ipython-input-2-129cae83a6fd>:49: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  weather[interpolation_cols] = weather[interpolation_cols].fillna(method='bfill')  # Fill remaining with backward filling


,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Destination Airport,Scheduled Departure Time,Actual Departure Time,Departure Delay (Minutes),Scheduled Departure Datetime,Rounded Datetime,Air Temperature,Dew Temperature,Sea Level Pressure,Wind Direction,Wind Speed Rate,Sky Condition,Precip 1 Hour
0,DL,01/01/2021,424.0,N672DL,ATL,06:00,06:00,0.0,2021-01-01 06:00:00,2021-01-01 06:00:00,0.221963,0.377119,0.820399,0.972222,0.182353,0.444444,0.0
1,DL,01/01/2021,703.0,N332NW,FLL,07:00,07:00,0.0,2021-01-01 07:00:00,2021-01-01 07:00:00,0.221963,0.387712,0.840355,1.000000,0.152941,0.222222,0.0
2,DL,01/01/2021,2063.0,N348NW,TPA,07:10,07:04,-6.0,2021-01-01 07:10:00,2021-01-01 07:00:00,0.221963,0.387712,0.840355,1.000000,0.152941,0.222222,0.0
3,DL,01/01/2021,428.0,N704X,SAN,07:30,07:27,-3.0,2021-01-01 07:30:00,2021-01-01 07:00:00,0.221963,0.387712,0.840355,1.000000,0.152941,0.222222,0.0
4,DL,01/01/2021,1065.0,N3748Y,MCO,07:35,07:28,-7.0,2021-01-01 07:35:00,2021-01-01 07:00:00,0.221963,0.387712,0.840355,1.000000,0.152941,0.222222,0.0


Data Preprocessing by Year (Only Weather)

In [ ]:
import pandas as pd
import glob

# Load weather data
weather = pd.read_csv('16_weather.csv')

# Convert weather date column to datetime
weather['Rounded Datetime'] = weather['date'].apply(custom_date_parser)

# Drop unnecessary columns (if any)
weather = weather.drop(columns=['date', 'station', 'precip_6hour'])  # Keep only relevant features

# Set negative precipitation values to 0
weather['precip_1hour'] = weather['precip_1hour'].apply(lambda x: max(x, 0))

# Fill missing numerical values using interpolation
interpolation_cols = ['air_temp', 'dew_temp', 'sea_level_pressure', 'wind_direction', 'wind_speed_rate', 'sky_condition', 'precip_1hour']
weather[interpolation_cols] = weather[interpolation_cols].interpolate(method='linear')  # Smooth filling
weather[interpolation_cols] = weather[interpolation_cols].fillna(method='bfill')  # Fill remaining with backward filling

# Normalize numerical columns using Min-Max scaling
weather[interpolation_cols] = (weather[interpolation_cols] - weather[interpolation_cols].min()) / (weather[interpolation_cols].max() - weather[interpolation_cols].min())

# Rename columns
weather.rename(columns={'air_temp': 'Air Temperature', 'dew_temp': 'Dew Temperature', 'sea_level_pressure': 'Sea Level Pressure', 'wind_direction': 'Wind Direction', 'wind_speed_rate': 'Wind Speed Rate', 'sky_condition': 'Sky Condition', 'precip_1hour': 'Precip 1 Hour'}, inplace=True)

# Save the final merged data
weather.to_csv('2016_weather.csv', index=False)

# Display first few rows
weather.head(48)

<ipython-input-11-2603a455e9fb>:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  weather[interpolation_cols] = weather[interpolation_cols].fillna(method='bfill')  # Fill remaining with backward filling


,Air Temperature,Dew Temperature,Sea Level Pressure,Wind Direction,Wind Speed Rate,Sky Condition,Precip 1 Hour,Rounded Datetime
0,0.478424,0.494755,0.561024,0.833333,0.268421,0.666667,0.0,2016-01-01 00:00:00
1,0.478424,0.475524,0.566929,0.861111,0.242105,0.740741,0.0,2016-01-01 01:00:00
2,0.469043,0.465035,0.568898,0.861111,0.378947,0.814815,0.0,2016-01-01 02:00:00
3,0.457786,0.456294,0.566929,0.888889,0.405263,0.888889,0.0,2016-01-01 03:00:00
4,0.448405,0.445804,0.561024,0.861111,0.352632,0.814815,0.0,2016-01-01 04:00:00
5,0.448405,0.456294,0.561024,0.861111,0.300000,0.740741,0.0,2016-01-01 05:00:00
6,0.437148,0.445804,0.547244,0.888889,0.300000,0.666667,0.0,2016-01-01 06:00:00
7,0.437148,0.445804,0.547244,0.888889,0.326316,0.740741,0.0,2016-01-01 07:00:00
8,0.427767,0.445804,0.547244,0.861111,0.215789,0.814815,0.0,2016-01-01 08:00:00
9,0.437148,0.445804,0.545276,0.777778,0.189474,0.888889,0.0,2016-01-01 09:00:00


Concatenation of Preprocessed Files

In [ ]:
import pandas as pd
import glob

# Path
path = '*.csv'

# Load and sort CSV files
csv_files = sorted(glob.glob(path))
dfs = []

# Process each CSV
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Combine all data
combined = pd.concat(dfs, ignore_index=True)

# Save the final merged data
combined.to_csv('2016-2021.csv', index=False)

# Display first few rows
combined.head()

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Destination Airport,Scheduled Departure Time,Actual Departure Time,Departure Delay (Minutes),Scheduled Departure Datetime,Rounded Datetime,Air Temperature,Dew Temperature,Sea Level Pressure,Wind Direction,Wind Speed Rate,Sky Condition,Precip 1 Hour
0,DL,01/01/2016,446.0,N900PC,ATL,06:00,05:59,-1.0,2016-01-01 06:00:00,2016-01-01 06:00:00,0.437148,0.445804,0.547244,0.888889,0.300000,0.666667,0.0
1,DL,01/01/2016,462.0,N37700,SEA,07:00,06:59,-1.0,2016-01-01 07:00:00,2016-01-01 07:00:00,0.437148,0.445804,0.547244,0.888889,0.326316,0.740741,0.0
2,DL,01/01/2016,420.0,N712TW,SLC,07:00,07:06,6.0,2016-01-01 07:00:00,2016-01-01 07:00:00,0.437148,0.445804,0.547244,0.888889,0.326316,0.740741,0.0
3,DL,01/01/2016,2217.0,N374DA,SJU,07:00,06:57,-3.0,2016-01-01 07:00:00,2016-01-01 07:00:00,0.437148,0.445804,0.547244,0.888889,0.326316,0.740741,0.0
4,DL,01/01/2016,2825.0,N396DA,MIA,07:00,07:00,0.0,2016-01-01 07:00:00,2016-01-01 07:00:00,0.437148,0.445804,0.547244,0.888889,0.326316,0.740741,0.0
